In [ ]:
# Stop warnings
# -------------
import warnings
warnings.filterwarnings("ignore")

# General imports
# ---------------
import os
import sys
import json
import glob
import numpy as np
import h5py
import scipy.io
opj = os.path.join
deb = ipdb.set_trace

# MRI imports
# -----------
import nibabel as nb
import cortex

# Functions import
# ----------------
from plot_class import PlotOperator
from utils import set_pycortex_config_file

# Plotly

# Get inputs
# ----------
# subject = sys.argv[1]
# task = sys.argv[2]
# preproc = sys.argv[3]
# save_svg = int(sys.argv[4])

# Define analysis parameters
# --------------------------
with open('settings.json') as f:
    json_s = f.read()
    analysis_info = json.loads(json_s)

# Define folders and settings
# ---------------------------
base_dir = analysis_info['base_dir']
rois = analysis_info['rois']
sample_ratio = analysis_info['sample_ratio']
deriv_dir = "{base_dir}/pp_data/{subject}/gauss/deriv".format(base_dir = base_dir, subject = subject)
h5_dir = "{base_dir}/pp_data/{subject}/gauss/h5".format(base_dir = base_dir, subject = subject)
bokeh_dir = "{base_dir}/pp_data/{subject}/gauss/bokeh_outputs".format(base_dir = base_dir, subject = subject)
try: os.makedirs(bokeh_dir)
except: pass

# Draw main analysis figure
# -------------------------
print('creating bokeh plots')
rsq_idx, ecc_idx, polar_real_idx, polar_imag_idx , size_idx, \
    amp_idx, baseline_idx, cov_idx, x_idx, y_idx, hemi_idx = 0,1,2,3,4,5,6,7,8,9,10

# Initialize data
# ---------------
for roi_num, roi in enumerate(rois):
    
    # create html folder
    exec("html_dir = '{bokeh_dir}/html/{task}_{preproc}/'".format(bokeh_dir = bokeh_dir, task = task, preproc = preproc))
    try: os.makedirs(html_dir)
    except: pass

    # create svg folder
    if save_svg == 1:
        exec("svg_dir = '{bokeh_dir}/svg/'".format(bokeh_dir = bokeh_dir))
        try: os.makedirs(svg_dir)
        except: pass

    # load h5 file
    h5_file = h5py.File("{h5_dir}/{roi}_{task}_{preproc}.h5".format(h5_dir = h5_dir, roi = roi, task = task, preproc = preproc),'r')

    # load deriv data
    deriv_data = h5_file['{folder_alias}/derivatives'.format(folder_alias = 'pRF')]
    
    # load time course data
    tc_data = h5_file['{folder_alias}/time_course'.format(folder_alias = 'pRF')]

    # load model time course data
    tc_model_data = h5_file['{folder_alias}/time_course_model'.format(folder_alias = 'pRF')]

    # load coordinates data
    coord_data = h5_file['{folder_alias}/coord'.format(folder_alias = 'pRF')]

    # threshold data
    voxel_num_ini = deriv_data.shape[0]
    voxel_num = 0
    if voxel_num_ini > 0:
        # take out nan
        tc_data = tc_data[~np.isnan(deriv_data[:,rsq_idx]),:]
        tc_model_data = tc_model_data[~np.isnan(deriv_data[:,rsq_idx]),:]
        coord_data = coord_data[~np.isnan(deriv_data[:,rsq_idx]),:]
        deriv_data = deriv_data[~np.isnan(deriv_data[:,rsq_idx]),:]

        # threshold on eccentricity and size
        deriv_data_th = deriv_data
        rsqr_th = deriv_data_th[:,rsq_idx] >= analysis_info['rsqr_th']
        size_th_down = deriv_data_th[:,size_idx] >= analysis_info['size_th'][0]
        size_th_up = deriv_data_th[:,size_idx] <= analysis_info['size_th'][1]
        ecc_th_down = deriv_data_th[:,ecc_idx] >= analysis_info['ecc_th'][0]
        ecc_th_up = deriv_data_th[:,ecc_idx] <= analysis_info['ecc_th'][1]
        all_th = np.array((rsqr_th,size_th_down,size_th_up,ecc_th_down,ecc_th_up)) 

        deriv_data = deriv_data[np.logical_and.reduce(all_th),:]
        tc_data = tc_data[np.logical_and.reduce(all_th),:]
        tc_model_data = tc_model_data[np.logical_and.reduce(all_th),:]
        coord_data = coord_data[np.logical_and.reduce(all_th),:]
        voxel_num = deriv_data.shape[0]
        
    # Draw and save figures
    # ---------------------
    if voxel_num > 0:
        # randomize order and take 10 %
        new_order = np.random.permutation(voxel_num)
        deriv_data = deriv_data[new_order,:]
        tc_data = tc_data[new_order,:]
        tc_model_data = tc_model_data[new_order,:]
        coord_data = coord_data[new_order,:]
        deriv_data_sample = deriv_data[0:int(np.round(voxel_num*sample_ratio)),:]

        print("drawing {roi}_{task}_{preproc} figures, n = {voxel_num}".format(roi = roi, voxel_num = voxel_num, task = task, preproc = preproc)) 

        data_source = { 'rsq': deriv_data[:,rsq_idx],
                        'ecc': deriv_data[:,ecc_idx],
                        'sigma': deriv_data[:,size_idx],
                        'beta': deriv_data[:,amp_idx],
                        'baseline': deriv_data[:,baseline_idx],
                        'cov': deriv_data[:,cov_idx],
                        'x': deriv_data[:,x_idx],
                        'y': deriv_data[:,y_idx],
                        'colors_ref': deriv_data[:,ecc_idx]}
        
        data_source_sample = { 
                        'rsq': deriv_data_sample[:,rsq_idx],
                        'ecc': deriv_data_sample[:,ecc_idx],
                        'sigma': deriv_data_sample[:,size_idx],
                        'beta': deriv_data_sample[:,amp_idx],
                        'baseline': deriv_data_sample[:,baseline_idx],
                        'cov': deriv_data_sample[:,cov_idx],
                        'x': deriv_data_sample[:,x_idx],
                        'y': deriv_data_sample[:,y_idx],
                        'colors_ref': deriv_data_sample[:,ecc_idx]}

        param_all = {   'roi_t': roi, 
                        'p_width': 400, 
                        'p_height': 400, 
                        'min_border_large': 10, 
                        'min_border_small': 5,
                        'bg_color': tuple([229,229,229]), 
                        'stim_color': tuple([250,250,250]), 
                        'hist_fill_color': tuple([255,255,255]),
                        'hist_line_color': tuple([0,0,0]), 
                        'stim_height': analysis_info['stim_height'],
                        'stim_width': analysis_info['stim_width'],
                        'cmap': 'Spectral',
                        'cmap_steps': 15,
                        'col_offset': 0,
                        'vmin': 0,
                        'vmax': 15,
                        'leg_xy_max_ratio': 1.8,
                        'dataMat': deriv_data,
                        'data_source': data_source,
                        'data_source_sample': data_source_sample,
                        'hist_range': (0,0.5),
                        'hist_steps': 0.5,
                        'h_hist_bins': 16,
                        'link_x': False,
                        'link_y': False,
                        'save_svg': save_svg
                        }

        if save_svg == 1:
            param_all.update({'svg_folder': svg_dir})

        plotter = PlotOperator(**param_all)